In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tf2onnx
import joblib

# Load dataset
df = pd.read_csv("Dataset.csv")

# Fix label range: Map Stress Level to 0-based integers
unique_labels = sorted(df['Stress Level'].unique())
label_map = {label: idx for idx, label in enumerate(unique_labels)}
inv_label_map = {v: k for k, v in label_map.items()}
df['Stress Level'] = df['Stress Level'].map(label_map)

# Features and target
X = df[['Soil moisture', 'Temperature', 'Humidity']]
y = df['Stress Level']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

# Build neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(3,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(unique_labels), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=16,
    verbose=1
)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {accuracy:.4f}")

# Convert to ONNX
spec = (tf.TensorSpec((None, 3), tf.float32, name="input"),)
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)

# Save ONNX model
with open("stress_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())
print(" Saved ONNX model as 'stress_model.onnx'")

# Save scaler and label map
joblib.dump(scaler, "scaler.save")
joblib.dump(label_map, "label_map.save")
print("Saved scaler and label map for inference")


Epoch 1/50
180/180 [==============================] - 1s 5ms/step - loss: 1.0891 - accuracy: 0.5628 - val_loss: 0.7987 - val_accuracy: 0.6906
Epoch 2/50
180/180 [==============================] - 1s 4ms/step - loss: 0.8418 - accuracy: 0.6417 - val_loss: 0.7239 - val_accuracy: 0.7156
Epoch 3/50
180/180 [==============================] - 1s 4ms/step - loss: 0.7928 - accuracy: 0.6490 - val_loss: 0.6883 - val_accuracy: 0.7375
Epoch 4/50
180/180 [==============================] - 1s 4ms/step - loss: 0.7567 - accuracy: 0.6677 - val_loss: 0.6887 - val_accuracy: 0.7531
Epoch 5/50
180/180 [==============================] - 1s 4ms/step - loss: 0.7258 - accuracy: 0.6774 - val_loss: 0.6393 - val_accuracy: 0.7469
Epoch 6/50
180/180 [==============================] - 1s 4ms/step - loss: 0.7077 - accuracy: 0.6913 - val_loss: 0.6062 - val_accuracy: 0.7500
Epoch 7/50
180/180 [==============================] - 1s 4ms/step - loss: 0.6745 - accuracy: 0.7024 - val_loss: 0.5903 - val_accuracy: 0.7625
Epoch 

2025-04-06 16:48:59.011916: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2025-04-06 16:48:59.012109: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2025-04-06 16:48:59.014608: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22281 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6
2025-04-06 16:48:59.179208: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22281 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6
2025-04-06 16:48:59.187475: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2025-04-06 16:48:59.187664: I tensorflow/core/grappler/clusters/single_machine.cc:

✅ Saved ONNX model as 'stress_model.onnx'
✅ Saved scaler and label map for inference
